In [1]:
import pandas as pd
import imblearn
import seaborn as sns
import BayesLib as bl
import os

from sklearn.datasets import make_classification
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *
from sklearn.feature_selection import SelectKBest, f_classif, chi2

from collections import Counter
from numpy import where
from matplotlib import pyplot
from imblearn.over_sampling import SMOTE
#from imblearn.over_sampling import SMOTENC
#from imblearn.over_sampling import BorderlineSMOTE 
#from imblearn.over_sampling import SVMSMOTE
#from imblearn.over_sampling import ADASYN 
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [2]:
#abriendo el conjunto de datos desde un archivo CSV y asignandolo los datos al DataFrame "df"
df = pd.read_csv('dataset_b.csv', sep=';', error_bad_lines=False)

In [3]:
%%capture
#Eliminando variables que no aportan al análisis
df = df.drop([
    'rut', 'usuario', 'correo', 'nombre', 'profesor', 
    'i1', 'f1', 's1', 
    'i2', 'f2', 's2',
    'i3', 'f3', 's3', 
    'i4', 'f4', 's4', 
    'i5', 'f5', 's5', 
    'i6', 'f6', 's6',
    'sol1', 'sol2', 'sol3', 'sol4',
    'tarea1', 'tarea2',
    'controles', 'np', 'examen'], axis=1)

In [4]:
#Estadísticas de la variable "programa"
serie = df['programa'].value_counts(normalize=True, ascending=True)*100
print(serie)

#Transformación numérica de la variable "programa"
for i in range(len(serie)):
    df["programa"] = df["programa"].str.replace(serie.index[i], str(i+1))
  
serie = df['programa'].value_counts(normalize=True, ascending=True)*100
print(serie)

df['programa'] = df['programa'].astype('int')

INGENIERIA INDUSTRIAL                       8.080808
INGENIERIA CIVIL INFORMATICA               14.141414
INGENIERIA EN COMPUTACION E INFORMATICA    37.373737
INGENIERIA CIVIL INDUSTRIAL                40.404040
Name: programa, dtype: float64
1     8.080808
2    14.141414
3    37.373737
4    40.404040
Name: programa, dtype: float64


In [5]:
#Estadísticas de la variable "estado"
serie = df['estado'].value_counts(normalize=True, ascending=False)*100
print(serie)

#Transformacíon numérica de la variable "estado"
for i in range(len(serie)):
    df["estado"] = df["estado"].str.replace(serie.index[i], str(i))

serie = df['estado'].value_counts(normalize=True, ascending=False)*100
print(serie)

df['estado'] = df['estado'].astype('int')

A    75.757576
R    24.242424
Name: estado, dtype: float64
0    75.757576
1    24.242424
Name: estado, dtype: float64


In [6]:
df = df.loc[:, ['lt', 'tt', 'pt', 'game_score', 
                'op1','op2', 'op3', 'op4', 'op5', 'op6',
                'sv1','sv2','sv3','sv4','sv5','sv6',
                'score', 'score_a', 'score_p', 'score_d', 'score_s',
                'programa', 'final', 'estado']]

In [7]:
binsNotas = 7
binsOtros = 5

valor_discreto = KBinsDiscretizer(n_bins=binsOtros, encode='ordinal', strategy = "kmeans").fit_transform(df[['lt']])
df[['lt']] = valor_discreto

valor_discreto = KBinsDiscretizer(n_bins=binsOtros, encode='ordinal', strategy = "kmeans").fit_transform(df[['tt']])
df[['tt']] = valor_discreto

puntaje_juego =  KBinsDiscretizer(n_bins=binsOtros, encode='ordinal', strategy = "kmeans").fit_transform(df[['pt']])
df[['pt']] = valor_discreto

puntaje_juego =  KBinsDiscretizer(n_bins=binsOtros, encode='ordinal', strategy = "kmeans").fit_transform(df[['game_score']])
df[['game_score']] = valor_discreto

nota_final =  KBinsDiscretizer(n_bins=binsNotas, encode='ordinal', strategy = "kmeans").fit_transform(df[['final']])
df[['final']] = valor_discreto

In [8]:
#indica cual va a ser el muestreo estratificado usando la clase "estado"
#cada fold mantiene la proporcion orignal de clases
#n_splits = el numero de experimentos a realizar
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1) 
target = df.loc[:, 'estado'] #todas las filas de la columna "estado"

fold_no = 1
for train_index, test_index in skf.split(df, target):
    #---------------------------------------------------------------------------
    #INICIO: SECCION DE ENTRENAMIENTO
    #---------------------------------------------------------------------------
    print("INICIO DE SECCION DE ENTRENAMIENTO, FOLD: ", str(fold_no))
    
    #entrega la porción de datos que serán usados como entrenamiento
    train = df.loc[train_index,:] #todas las columnas de la fila "train_index"
         
    #aprendiendo la estructura y los parametros de la porción de datos entrenados "sobre muestrados"
    modelo = bl.Aprendizaje(train, fold_no, "TRAIN")
    
    #transformando el modelo aprendido en un dataset que pueda ser inferido
    newModel = bl.modelToDataFrame(modelo, train)

    #realizando la inferencia de los datos de entrenamiento
    probTrain = bl.probabilidadConjunta(modelo, newModel, fold_no, "TRAIN")
            
    i = 0 #columna que queremos obtener
    lista_train = [fila[i] for fila in probTrain]

    #Metricas finales TRAIN
    varAccuracy = accuracy_score(lista_train, train.loc[:, 'estado'])
    varBalancedAccuracy = balanced_accuracy_score(lista_train, train.loc[:, 'estado'])
    varPrecision = precision_score(lista_train, train.loc[:, 'estado'], average='weighted')
    varRecall = recall_score(lista_train, train.loc[:, 'estado'], average='weighted')
    varRocAuc = roc_auc_score(train.loc[:, 'estado'], lista_train, multi_class='ovr')
    varRatio = sum(train['estado'])/len(train['estado'])
    
    print('RESULTADOS DEL ENTRENAMIENTO CON SOBREMUESTREO:')
    print('===============================================')
    print('(TRAIN) Fold', str(fold_no), 'Accuracy          :', str(varAccuracy))  
    print('(TRAIN) Fold', str(fold_no), 'Balanced Accuracy :', str(varBalancedAccuracy))  
    print('(TRAIN) Fold', str(fold_no), 'Precision Score   :', str(varPrecision))  
    print('(TRAIN) Fold', str(fold_no), 'Recall Score      :', str(varRecall))  
    print('(TRAIN) Fold', str(fold_no), 'ROC AUC           :', str(varRocAuc)) 
    print('(TRAIN) Fold', str(fold_no), 'Class Ratio       :', str(varRatio))
    
    #Guardando las metricas en un archivo
    file = open('Metricas_'+str(fold_no)+'.txt', 'w')
    file.write('RESULTADOS DEL ENTRENAMIENTO CON SOBREMUESTREO:' + os.linesep)
    file.write('===============================================' + os.linesep)
    file.write('(TRAIN) Fold', str(fold_no), 'Accuracy          :', str(varAccuracy) + os.linesep)  
    file.write('(TRAIN) Fold', str(fold_no), 'Balanced Accuracy :', str(varBalancedAccuracy) + os.linesep)  
    file.write('(TRAIN) Fold', str(fold_no), 'Precision Score   :', str(varPrecision) + os.linesep)  
    file.write('(TRAIN) Fold', str(fold_no), 'Recall Score      :', str(varRecall) + os.linesep)  
    file.write('(TRAIN) Fold', str(fold_no), 'ROC AUC           :', str(varRocAuc) + os.linesep) 
    file.write('(TRAIN) Fold', str(fold_no), 'Class Ratio       :', str(varRatio) + os.linesep)
    file.close()
    
    
    print("FIN DE SECCION DE ENTRENAMIENTO, FOLD: ", str(fold_no))
    #---------------------------------------------------------------------------
    #FIN: SECCION DE ENTRENAMIENTO
    #---------------------------------------------------------------------------

    #---------------------------------------------------------------------------
    #INICIO: SECCION DE PRUEBAS
    #---------------------------------------------------------------------------
    print("INICIO DE SECCION DE PRUEBAS, FOLD: ", str(fold_no))
    
    #entrega la porción de datos que serán usados como pruebas
    test = df.loc[test_index,:] #todas las columnas de la fila "test_index"
        
    #aprendiendo la estructura y los parametros de la porción de datos de pruebas
    modelo = bl.Aprendizaje(test, fold_no, "TEST")

    #transformando el modelo aprendido en un dataset que pueda ser inferido
    newModel = bl.modelToDataFrame(modelo, test)
    
    #realizando la inferencia de los datos de prueba
    probTest = bl.probabilidadConjunta(modelo, newModel, fold_no, "TEST")
    
    ##i = 0 #columna que queremos obtener
    lista_test = [fila[i] for fila in probTest]
    
    #Metricas finales TEST
    varAccuracy = accuracy_score(lista_test, test.loc[:, 'estado'])
    varBalancedAccuracy = balanced_accuracy_score(lista_test, test.loc[:, 'estado'])
    varPrecision = precision_score(lista_test, test.loc[:, 'estado'], average='weighted')
    varRecall = recall_score(lista_test, test.loc[:, 'estado'], average='weighted')
    varRocAuc = roc_auc_score(test.loc[:, 'estado'], lista_test, multi_class='ovr')
    varRatio = sum(test['estado'])/len(test['estado'])

    print('RESULTADOS DEL TEST:')
    print('===============================================')
    print('(TEST) Fold', str(fold_no), 'Accuracy          :', str(varAccuracy))  
    print('(TEST) Fold', str(fold_no), 'Balanced Accuracy :', str(varBalancedAccuracy))  
    print('(TEST) Fold', str(fold_no), 'Precision Score   :', str(varPrecision))  
    print('(TEST) Fold', str(fold_no), 'Recall Score      :', str(varRecall))  
    print('(TEST) Fold', str(fold_no), 'ROC AUC           :', str(varRocAuc)) 
    print('(TEST) Fold', str(fold_no), 'Class Ratio       :', str(varRatio))

    #Guardando las metricas en un archivo
    file = open('Metricas_'+str(fold_no)+'.txt', 'a')
    file.write('RESULTADOS DEL TEST:' + os.linesep)
    file.write('===============================================' + os.linesep)
    file.write('(TEST) Fold', str(fold_no), 'Accuracy          :', str(varAccuracy) + os.linesep)  
    file.write('(TEST) Fold', str(fold_no), 'Balanced Accuracy :', str(varBalancedAccuracy) + os.linesep)  
    file.write('(TEST) Fold', str(fold_no), 'Precision Score   :', str(varPrecision) + os.linesep)  
    file.write('(TEST) Fold', str(fold_no), 'Recall Score      :', str(varRecall) + os.linesep)  
    file.write('(TEST) Fold', str(fold_no), 'ROC AUC           :', str(varRocAuc) + os.linesep) 
    file.write('(TEST) Fold', str(fold_no), 'Class Ratio       :', str(varRatio) + os.linesep)

    print("FIN DE SECCION DE PRUEBAS, FOLD: ", str(fold_no))
    #---------------------------------------------------------------------------
    #FIN: SECCION DE PRUEBAS
    #---------------------------------------------------------------------------

    #Cambiando de fold
    break
    #fold_no += 1

SyntaxError: invalid syntax (<ipython-input-8-e3e9d54a0bb2>, line 24)